<a href="https://colab.research.google.com/github/anuragsaraf1912/Babbar450/blob/master/getMarketData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Data Analysis

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output

#Getting the widgets objects
import ipywidgets as widgets
from ipywidgets import HBox, VBox

## Functions

#### Graph Generating Functions

In [58]:
def plotCandleStick(data):
  """Plots candlestick pattern using the Plotly library"""

  candlestick = go.Candlestick(
    x=data["Date"],
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    name="Price",
    increasing_line_color='green',
    decreasing_line_color='red',
    hovertext=[f"Returns: {round(row['Daily Returns'],2)}% <br> Volume: {round(row['Volume']/1000,2)}K <br> Open: {round(row['Open'],2)} <br> High: {round(row['High'],2)} <br> Low: {round(row['Low'],2)} <br> Close: {round(row['Close'],2)}" for index, row in data.iterrows()],
    hoverinfo="text"  # Display hovertext
  )

  # Create volume trace
  volume = go.Bar(
      x=data["Date"],
      y=data["Volume"],
      name="Volume",
      marker=dict(color='rgba(0, 0, 255, 0.3)')
  )

  # Create figure

  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(
      candlestick,
      secondary_y=False,
  )

  fig.add_trace(
      volume,
      secondary_y=True,
  )

  # Update layout
  fig.update_layout(
      title= f"Candlestick Chart for {dropdown.label} with Volume",
      height = 1000,
      width = 2000,
      xaxis=dict(
          title="Date",
      ),
  )

  # Configure the y-axis for volume
  fig.update_yaxes(title_text="Price", secondary_y=False, )
  fig.update_yaxes(title_text="Volume", secondary_y=True, range =[0, 5*max(data["Volume"])])

  # Show figure
  fig.show()

###################################################################################################################################################################

def makeHistogram(df):
  """Creates a histogram from the data. Requires a 'Daily Returns' column"""
  fig = go.Figure(data=[go.Histogram(x=df['Daily Returns'])])

  # Update layout for better appearance
  fig.update_layout(
      title= f"Daily Returns Distribution for {dropdown.label}",
      xaxis_title="Returns",
      yaxis_title="Count",
      bargap=0.1,  # Gap between bars
      bargroupgap=0.1,  # Gap between groups of bars
      height = 700,
      width = 1000
  )

  # Show the figure (interactive)
  fig.show()

###################################################################################################################################################################

def makeLinePlot(sortedDF):
  """Create a quartle graph from the returns data. Requires a percentile and Daily returns column"""

  # Create a line graph using go.Scatter
  lineGraph = go.Figure()

  x_values, y_values = round(sortedDF['Daily Returns'],3),  round(sortedDF['percentile'],3)

  # Add the line graph trace
  lineGraph.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Percentile Line'))


  # Customize the layout
  lineGraph.update_layout(
      title= f"Percentile line for the Daily Returns for {dropdown.label}",
      xaxis_title="Daily Returns",
      yaxis_title="Percentile",
      height=700,
      width=1000,
  )
  # Show the figure
  lineGraph.show()

###################################################################################################################################################################

def makeTimeSeries(df):
  """Create a time series graph from the data. Requires a 'Daily Returns' and 'Date' columns"""
  fig = go.Figure()

  # Add the returns line
  fig.add_trace(go.Scatter(
      x=df['Date'],
      y=round(df['Daily Returns'],3),
      mode='lines',
      name='Daily Returns'#,
      # hovertext=[f"Date: {date.date()}<br>Return: {return_val:.2%}" for date, return_val in zip(df['Date'], df['Returns'])],
      # hoverinfo='text'
  ))

  # Customize the layout
  fig.update_layout(
      title= f"Time Series of Daily Returns for {dropdown.label}",
      xaxis_title="Date",
      yaxis_title="Returns",
      xaxis_rangeslider_visible=False,
      height = 700,
      width = 1000
  )

  # Add a horizontal dashed line at y=0
  fig.add_shape(
      type='line',
      x0=df['Date'].iloc[0],  # Starting x value (beginning of the x-axis)
      y0=0,                   # y value (height of the line)
      x1=df['Date'].iloc[-1], # Ending x value (end of the x-axis)
      y1=0,                   # y value (height of the line)
      line=dict(color='red', width=1, dash='dash')  # Line style
  )

  # Show the figure
  fig.show()


#### Data Logic

In [66]:
def getTicker(tick):
  """Logic to modify ticker before sending it to the Yahoo finance API"""
  if tick in ['^NSEI', '^BSESN', '^NSEBANK']:
    return tick
  else:
    return tick + ".NS"

def getData(ticker, startDate, endDate):
  """Logic to get data from the Yahoo finance API"""
  df = (yf.download(ticker, startDate, endDate)).reset_index()
  return df

def getDailyReturns(df):
  """Logic to get daily returns from the data"""
  df['Daily Returns'] = 100*np.log(df['Adj Close']/df['Adj Close'].shift(1))
  df.dropna(inplace=True)
  return df

def genSortedPercentile(df):
  """Logic to generate the percentile column"""
  sortedDF = df.sort_values('Daily Returns')
  sortedDF['percentile'] =  1
  sortedDF['percentile'] = sortedDF['percentile'].cumsum()/len(sortedDF)
  return sortedDF


#### Event functions

In [4]:
def on_value_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        startDate = start_date_picker.value
        endDate = end_date_picker.value
        ticker = getTicker(dropdown.value)
        if startDate and endDate and ticker:
          global df
          df = getData(ticker, startDate, endDate)
          df = getDailyReturns(df)
          global sortedDF
          sortedDF = genSortedPercentile(df)
          clear_output(wait = True)
          display(widgets.HBox([dropdown, start_date_picker, end_date_picker])
)

## Widgets

#### Initiate Widgets (Add Tickers)

In [83]:
# Date Picker Widgets
start_date_picker = widgets.DatePicker(description='Start Date:')
end_date_picker = widgets.DatePicker(description='End Date:')

# Text Entry Widget
options = [('NIFTY50','^NSEI'), ('SENSEX','^BSESN'), ('BANKNIFTY','^NSEBANK'), ('HDFCBANK','HDFCBANK'),
          ('TCS','TCS'), ('RELIANCE', 'RELIANCE'), ('PERSISTENT', 'PERSISTENT')]

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=options,
    description='Select stock:'
)

# Observing changes
start_date_picker.observe(on_value_change)
end_date_picker.observe(on_value_change)
dropdown.observe(on_value_change)

#### Stock and Date Selector

In [72]:
# Displaying widgets
widgets.HBox([dropdown, start_date_picker, end_date_picker])

# Analysis

In [88]:
plotCandleStick(df)

In [79]:
makeLinePlot(sortedDF)

In [80]:
makeHistogram(df)

In [81]:
makeTimeSeries(df)

In [82]:
df['Daily Returns'].describe()

,Daily Returns
count,181.000000
mean,0.057268
std,1.153951
min,-8.280507
25%,-0.411137
50%,0.095001
75%,0.542980
max,4.430666
